In [1]:
import matplotlib.pyplot as plt
import koreanize_matplotlib
import pandas as pd
import seaborn as sns
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os
import re
import platform


In [4]:


# ▒ 한글 폰트 설정
if platform.system() == 'Darwin':
    plt.rcParams['font.family'] = 'AppleGothic'
elif platform.system() == 'Windows':
    plt.rcParams['font.family'] = 'Malgun Gothic'
else:
    plt.rcParams['font.family'] = 'NanumGothic'
plt.rcParams['axes.unicode_minus'] = False

# ▶️ .env 파일의 환경변수 불러오기
load_dotenv()
host = os.getenv("DB_HOST")
user = os.getenv("DB_USER")
password = os.getenv("DB_PASSWORD")
database = os.getenv("DB_NAME")

# ▶️ MySQL 연결
engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}:3306/{database}")

# ▒ 데이터 조회
query = """
SELECT road_address, jibun_address
FROM seoul_animal_hospital
WHERE status_name in ('영업/정상')  -- 폐업 제외
"""
df = pd.read_sql(query, engine)


# ▒ 자치구 추출 함수
def extract_district(address):
    # match = re.search(r'서울\s*(\S+구)', str(address))
    # return match.group(1) if match else None
    if address:
        return address.split(' ')[1]
    else:
        None

# ▒ 자치구 컬럼 생성
df['자치구'] = df['road_address'].apply(extract_district)
df['자치구'] = df['자치구'].fillna(df['jibun_address'].apply(extract_district))

# print(df)

# ▒ 병원 수 집계
district_count = df['자치구'].value_counts().reset_index()
district_count.columns = ['자치구', '병원 수']
district_count = district_count.sort_values(by='병원 수', ascending=False)

print(district_count)

     자치구  병원 수
0    강남구    94
1    송파구    83
2    강동구    59
3    서초구    53
4    강서구    50
5    양천구    49
6    마포구    47
7    노원구    45
8    은평구    42
9   영등포구    37
10   광진구    36
11   성북구    34
12  동대문구    32
13   중랑구    32
14   구로구    29
15   용산구    28
16   성동구    27
17   강북구    27
18   관악구    27
19  서대문구    27
20   도봉구    26
21   동작구    21
22    중구    19
23   금천구    16
24   종로구    12
